# BUILDING A LANGCHAIN AGENT
### ECUADORIAN CURRENT NEWS 

1. Crear un chain que realice una busqueda personalizada en cinco periódicos importantes del Ecuador, y según eso responda
- https://www.expreso.ec/
- https://www.primicias.ec/
- https://www.elcomercio.com/
- https://www.lahora.com.ec/
- https://www.eluniverso.com/

In [63]:
# Empezamos importando las librerías necesarias para el proyecto
from dotenv import load_dotenv
import os
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from langchain import hub

In [62]:
# Nos aseguramos de que las variables de entorno necesarias estén cargadas,
# las cuales deben de estar previamente establecidas en un archivo .env
# Para entender un poco mas de las keys, dirigete a:
# GOOGLE_CSE_ID: https://programmablesearchengine.google.com/about/
# GOOGLE_API_KEY: https://console.cloud.google.com/apis/credentials
# OPENAI_API_KEY: https://platform.openai.com/docs/api-reference/introduction

load_dotenv()

os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [17]:
# Luego instanciamos el wrapper de google search el cual nos permitirá acceer a nuestro custom google
# search engine para buscar información en la web.
search = GoogleSearchAPIWrapper()

# Definimos una función que nos permitirá buscar los 5 primeros resultados de una query en google
def top5_results(query):
    return search.results(query, 5)

# Creamos una instancia de la herramienta (tool) que nos permitirá buscar los 5 primeros resultados de una query
tool = Tool(
    name="ecuador_gsearch",
    description="Search Google for recent results about Ecuador.",
    func=top5_results,
)

# Corremos un query de prueba
gsearch_result = tool.run("¿Cómo va la busqueda de los 4 ninos desaparecidos?")
print(gsearch_result)

# Ahora bien, vamos a buscar la info de estos urls usando los loaders de langchain
urls = [r['link'] for r in gsearch_result]
loader = UnstructuredURLLoader(urls=urls)
docs = loader.load()
print(docs)

In [68]:
# Para poder usar todo esto dentro de un chain de langchain, necesitamos crear una función 
# que nos permita realizar los pasos anteriores en una sola llamada

def custom_gsearch(query):
    # Busqueda de google
    gsearch_result = tool.run(query)
    # Obtener la info de los urls
    urls = [r['link'] for r in gsearch_result]
    loader = UnstructuredURLLoader(urls=urls)
    docs = loader.load()
    # Unir la info de los docs
    docs = [d.page_content.replace("\n\n", " ") for d in docs]
    docs_join = "\n\n".join(docs)
    return docs_join

In [69]:
# Hacemos un retrieve desde el langchain prompt hub para obtener el prompt que nos permitirá
# contestar la pregunta dado el contexto que hemos obtenido de la busuqeda de google
prompt = hub.pull("rlm/rag-prompt")

# Cargar el modelo de OpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)

# Definimos el chain con el pipeline integrado
chain = (
        {"context": custom_gsearch, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )

c:\Users\kpam2\OneDrive\Documents\2025\EcuadorSecurity\venv\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [70]:
# response = chain.invoke("¿Cómo está la situación de seguridad en el Ecuador?")
# response = chain.invoke("¿Cómo va la busqueda de los 4 ninos desaparecidos?")
response = chain.invoke("¿Se encontraron ya a los 4 ninos desaparecidos?")
print(response)

Sí, los cuerpos de los cuatro niños desaparecidos fueron hallados en Taura, con pruebas de ADN en proceso para confirmar su identidad. Las familias de los menores están esperando confirmación sobre la identidad de los cuerpos encontrados. Los padres de los niños desaparecidos claman por justicia y esperan que se esclarezca lo ocurrido.


2. Crear un agente de langchain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

In [86]:
# Crear una instancia de OpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Luego instanciamos el wrapper de google search el cual nos permitirá acceer a nuestro custom google
# search engine para buscar información en la web.
search = GoogleSearchAPIWrapper()

# Definimos una función que nos permitirá buscar los 5 primeros resultados de una query en google
def top5_results(query):
    return search.results(query, 5)

# Creamos una instancia de la herramienta (tool) que nos permitirá buscar los 5 primeros resultados de una query
gsearch_tool = Tool(
        name="ecuador_gsearch",
        description="Search Google for recent results about Ecuador.",
        func=top5_results,
    )

# Creamos una función con el decorador tool que nos permitirá luego
# unirla al modelo, para que el agente pueda decidir cuando usarla
@tool
def tool_gsearch(query:str) -> str:
    """Search Google for recent results about Ecuador."""
    # Busqueda de google
    gsearch_result = gsearch_tool.run(query)
    # Obtener la info de los urls
    urls = [r['link'] for r in gsearch_result]
    loader = UnstructuredURLLoader(urls=urls)
    docs = loader.load()
    # Unir la info de los docs
    docs = [d.page_content.replace("\n\n", " ") for d in docs]
    docs_join = "\n\n".join(docs)
    return docs_join

In [93]:
# Creamos un prompt que nos permitirá enviar al modelo el query junto con un agent_scratchpad 
# que contendrá los mensajes de las invocaciones de las tools del modelo y sus outputs
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un excelente asistente, pero no sabes mucho de las noticias actuales de Ecuador",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Definimos los tools que vamos a usar en el modelo
tools = [tool_gsearch]
# Unimos los tools al modelo
llm_with_tools = llm.bind_tools(tools)

# Definimos el agente que recibirá un prompt, tomará la decisión de cuando usar la tool y luego
# usará las agent actions y los output tools para parsear el prompt que ingresará al modelo 
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# En este caso el agente decide usar la tool para buscar información en google
list(agent_executor.stream({"input": "¿Se encontraron ya a los 4 ninos desaparecidos?"}))



> Entering new None chain...

Invoking: `tool_gsearch` with `{'query': 'niños desaparecidos Ecuador'}`


Seguridad Estas son los cinco faltas y omisiones sobre la desaparición de los cuatro niños de Malvinas, en Guayaquil La audiencia de hábeas corpus en la que se declaró la desaparición forzada de cuatro niños reveló una serie de irregularidades de procedimiento, entre ellas que no existió un parte militar de la detención. Un letrero con una línea roja para situaciones de peligro señaliza la salida sur de la parroquia rural de Taura (Naranjal, Guayas), el 24 de diciembre de 2024. Se trata del último paradero conocido de cuatro menores desaparecidos y donde se hallaron cuatro cuerpos incinerados. Foto API Autor: Alexander García Actualizada: 26 dic 2024 - 16:55 Compartir: Unirse a Whatsapp El caso de los cuatro niños desaparecidos el 8 de diciembre tras su detención en el sur de Guayaquil deja en evidencia una serie de irregularidades en los procedimientos adoptados tras su captura p

[{'actions': [ToolAgentAction(tool='tool_gsearch', tool_input={'query': 'niños desaparecidos Ecuador'}, log="\nInvoking: `tool_gsearch` with `{'query': 'niños desaparecidos Ecuador'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_MxKDuje1Q1btaWa9sU3L4lCU', 'function': {'arguments': '{"query":"niños desaparecidos Ecuador"}', 'name': 'tool_gsearch'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-316a9168-b048-482a-88fe-69bc87fc8584', tool_calls=[{'name': 'tool_gsearch', 'args': {'query': 'niños desaparecidos Ecuador'}, 'id': 'call_MxKDuje1Q1btaWa9sU3L4lCU', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'tool_gsearch', 'args': '{"query":"niños desaparecidos Ecuador"}', 'id': 'call_MxKDuje1Q1btaWa9sU3L4lCU', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_MxKDuje1Q1btaWa9sU3L4lCU')],
  'messages': [AIMessageChunk(content='', additional_kwarg

In [ ]:
# En este caso el agente decide usar la tool para buscar información en google
list(agent_executor.stream({"input": "¿Cómo va la busqueda de los 4 ninos desaparecidos?"}))



> Entering new None chain...

Invoking: `tool_gsearch` with `{'query': 'Búsqueda de los 4 niños desaparecidos en Ecuador'}`






Seguridad Los cuatro de Las Malvinas: Estado ecuatoriano debe informar las medidas adoptadas por desaparición de los niños hasta el 7 de enero El Comité contra la Desaparición Forzada de la ONU emitió recomendaciones al Estado ecuatoriano. La CIDH monitorea el caso. El Ministerio de Derechos Humanos responde y aduce tratamiento reservado. Padres de los cuatro niños desaparecidos en Guayaquil muestran las fotografías de los menores, durante un plantón en el centro de la ciudad. Foto AFP Autor: Redacción Primicias Actualizada: 24 dic 2024 - 17:15 Compartir: Unirse a Whatsapp El caso de los cuatro niños desaparecidos en Ecuador el 8 de diciembre, cuando una patrulla militar los aprehendió, ha dado la vuelta al mundo y ha captado la atención de los principales organismos de derechos humanos. La tarde del 23 de diciembre, los familiares de los cuatro niños de L

[{'actions': [ToolAgentAction(tool='tool_gsearch', tool_input={'query': 'Búsqueda de los 4 niños desaparecidos en Ecuador'}, log="\nInvoking: `tool_gsearch` with `{'query': 'Búsqueda de los 4 niños desaparecidos en Ecuador'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_wlQwNZBuRQOLDjAtore0i3Gy', 'function': {'arguments': '{"query":"Búsqueda de los 4 niños desaparecidos en Ecuador"}', 'name': 'tool_gsearch'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-c90079b9-c725-4040-83ac-705f1f529135', tool_calls=[{'name': 'tool_gsearch', 'args': {'query': 'Búsqueda de los 4 niños desaparecidos en Ecuador'}, 'id': 'call_wlQwNZBuRQOLDjAtore0i3Gy', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'tool_gsearch', 'args': '{"query":"Búsqueda de los 4 niños desaparecidos en Ecuador"}', 'id': 'call_wlQwNZBuRQOLDjAtore0i3Gy', 'index': 0, 'type': 'tool_call_chunk'}])], t

In [110]:
# En este caso el agente no usa la tool y responde directamente
list(agent_executor.stream({"input": "¿Cuanto es 44 por 89?"}))



> Entering new None chain...
El resultado de multiplicar 44 por 89 es 3916.

> Finished chain.


[{'output': 'El resultado de multiplicar 44 por 89 es 3916.',
  'messages': [AIMessage(content='El resultado de multiplicar 44 por 89 es 3916.', additional_kwargs={}, response_metadata={})]}]

In [ ]:
# En este caso el agente decide usar la tool para buscar información en google
list(agent_executor.stream({"input": "Quiero viajar manana a Machala, pero quiero un estado de los ultimos crimenes en Machala ¿Según el resultado crees que es seguro viajar hoy o no?"}))



> Entering new None chain...

Invoking: `tool_gsearch` with `{'query': 'últimos crímenes en Machala'}`


Sucesos Doble asesinato en Machala, uno era el objetivo y otro es víctima colateral Las dos víctimas se movilizaban en una vehículo en una de las vías principales, al sureste de Machala, en la provincia de El Oro. Personal de la Policía y de Criminalística toman procedimiento por el doble crimen registrado este miércoles 6 de noviembre de 2024 en Machala, provincia de El Oro. Foto Cortesía Autor: Redacción Primicias Actualizada: 06 nov 2024 - 18:54 Compartir: Unirse a Whatsapp A los ataques con bombas, robos y secuestros, en el cantón Machala, provincia de El Oro, las muertes violentas se cuentan a diario. Este 6 de noviembre, dos hombres que se movilizaban en un vehículo color negro por la vía Colón Tinoco fueron acribillados a plena luz del día. Este hecho violento se produjo luego de que una de las víctimas tuviera un altercado en un centro de tolerancia de la zona, confirmó Ju

[{'actions': [ToolAgentAction(tool='tool_gsearch', tool_input={'query': 'últimos crímenes en Machala'}, log="\nInvoking: `tool_gsearch` with `{'query': 'últimos crímenes en Machala'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_5oP1et1Ly9hVeLmHlp8HzHna', 'function': {'arguments': '{"query":"últimos crímenes en Machala"}', 'name': 'tool_gsearch'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-c0e34d12-14d2-43cf-b94c-6a06e5bce1e6', tool_calls=[{'name': 'tool_gsearch', 'args': {'query': 'últimos crímenes en Machala'}, 'id': 'call_5oP1et1Ly9hVeLmHlp8HzHna', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'tool_gsearch', 'args': '{"query":"últimos crímenes en Machala"}', 'id': 'call_5oP1et1Ly9hVeLmHlp8HzHna', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_5oP1et1Ly9hVeLmHlp8HzHna')],
  'messages': [AIMessageChunk(content='', additional_kwarg

In [ ]:
# En este caso el agente decide no usar la tool y responde directamente
list(agent_executor.stream({"input": "¿Cual es la capital de Italia?"}))



> Entering new None chain...
La capital de Italia es Roma. ¿Hay algo más en lo que pueda ayudarte?

> Finished chain.


[{'output': 'La capital de Italia es Roma. ¿Hay algo más en lo que pueda ayudarte?',
  'messages': [AIMessage(content='La capital de Italia es Roma. ¿Hay algo más en lo que pueda ayudarte?', additional_kwargs={}, response_metadata={})]}]